In [33]:
from pathlib import Path
import random
from tqdm.notebook import tqdm
from tabulate import tabulate
from collections import Counter
import pandas as pd
from omnibelt import load_yaml, save_yaml, load_json, save_json
import omnifig as fig
from yfinance import Ticker
from stox import misc, general
fig.initialize()

In [38]:
symbols = general.load_symbol_table()
yf2ib = {y: sym['ibkr-contract']['symbol'] for y, sym in symbols.items()}
ib2yf = {i:t for t, i in yf2ib.items()}

In [86]:
ib2yf['AMP2']

'AMP.MI'

In [3]:
root = misc.assets_root() / 'dis'

In [5]:
raw = load_yaml(root / 'revision4.yaml')
pf = {v['ticker']: v['weight'] for v in raw['tickers'] if v['weight'] > 0}
sum(pf.values())

99.99999999999999

In [39]:
ibkr = {key: yf2ib[key] for key in pf.keys()}
fe_current_raw = dict(misc.extract_tickers_and_shares(root / '..' / 'ibkr' / 'fe-aug23.txt'))
fe_current = {ib2yf[i]: s for i, s in fe_current_raw.items()}
gu_current_raw = dict(misc.extract_tickers_and_shares(root / '..' / 'ibkr' / 'gu-aug23.txt'))
gu_current = {ib2yf[i]: s for i, s in gu_current_raw.items()}
len(fe_current_raw), len(gu_current_raw)

(58, 45)

In [41]:
vocab = set(fe_current.keys()) | set(gu_current.keys()) | set(pf.keys())
len(vocab)

65

In [8]:
ytks = {t:Ticker(t) for t in pf.keys()}
prices = {t: y.info['currentPrice'] for t, y in tqdm(ytks.items())}

In [45]:
fe_total = sum(prices[t] * s for t, s in fe_current.items())
gu_total = sum(prices[t] * s for t, s in gu_current.items())

In [48]:
fe_only, gu_only, both, new = set(), set(), set(), set()
for t in pf.keys():
	if t in fe_current and t in gu_current:
		both.add(t)
	elif t in fe_current:
		fe_only.add(t)
	elif t in gu_current:
		gu_only.add(t)
	else:
		new.add(t)
len(fe_only), len(gu_only), len(both), len(new)

(11, 2, 34, 3)

In [14]:
budget = 100_000

In [53]:
ideal = {t: budget * w / 100 for t, w in pf.items()}

In [55]:
shares = {t: int(p / prices[t]) for t, p in ideal.items()}
actual = {t: shares[t] * prices[t] for t in pf.keys()}
err = {t: ideal[t] - actual[t] for t in pf.keys()}
errp = {t: 100 * err[t] / ideal[t] for t in pf.keys()}
sum(err.values())

2667.3230000000117

In [62]:
df = pd.DataFrame({'ibkr': ibkr, 'price': prices, 'shares': shares, 'ideal': ideal, 'err': err, 'errp': errp})
# df

In [103]:
fe = {t: actual[t] for t in fe_only}
gu = {t: actual[t] for t in gu_only}
# sum(fe.values()), sum(gu.values())

In [82]:
todo = set(pf.keys()) - set(fe.keys()) - set(gu.keys())
todo = list(todo)
random.shuffle(todo)
for t in todo:
	fe_update = sum(fe.values()) + actual[t]
	if fe_update < budget/2:
		fe[t] = actual[t]
	else:
		gu[t] = actual[t]
fe_total_new, gu_total_new = sum(fe.values()), sum(gu.values())

In [83]:
fe_delta = {t: shares[t] - fe_current.get(t,0) for t in fe.keys()}
fe_delta.update({t: -v for t, v in fe_current.items() if t not in fe_delta})
gu_delta = {t: shares[t] - gu_current.get(t,0) for t in gu.keys()}
gu_delta.update({t: -v for t, v in gu_current.items() if t not in gu_delta})

In [101]:
fe_todo = [{'ib': yf2ib[t], 'ticker': t, 'delta': v, 'price': prices[t], 'goal': pf.get(t,0), 'value_change': prices[t]*v, 'shares': shares.get(t,0), 'individual': 100*actual.get(t,0)/fe_total_new, 'current': fe_current.get(t,0), 'err': err.get(t,0), 'errp': errp.get(t,0)
			} for t, v in fe_delta.items() if v != 0]
fe_todo.sort(key=lambda x: (x['delta']<0, x['ticker'] in fe_only, abs(x['value_change']), x['individual']), reverse=True)
fe_df = pd.DataFrame(fe_todo, columns=['ib', 'delta', 'price', 'current', 'shares', 'goal', 'ticker', 'value_change', 'individual', 'err', 'errp'])
# fe_df

In [102]:
gu_todo = [{'ib': yf2ib[t], 'ticker': t, 'delta': v, 'price': prices[t], 'goal': pf.get(t,0), 'value_change': prices[t]*v, 'shares': shares.get(t,0), 'individual': 100*actual.get(t,0)/gu_total_new, 'current': gu_current.get(t,0), 'err': err.get(t,0), 'errp': errp.get(t,0)
			} for t, v in gu_delta.items() if v != 0]
gu_todo.sort(key=lambda x: (x['delta']<0, x['ticker'] in gu_only, abs(x['value_change']), x['individual']), reverse=True)
gu_df = pd.DataFrame(gu_todo, columns=['ib', 'delta', 'price', 'current', 'shares', 'goal', 'ticker', 'value_change', 'individual', 'err', 'errp'])
# gu_df

In [87]:
full = [{'ticker': t, 'price': prices[t], 'shares': shares[t], 'ideal': ideal[t], 'err': err[t], 'errp': errp[t]} for t in ideal.keys()]
fulldf = pd.DataFrame(full, columns=['ticker', 'price', 'shares', 'ideal', 'err', 'errp'])

In [97]:
shares.get(ib2yf['SAN1'], 0)

0